# Praktikum 4: Klasifikasi dengan ANN

## Deskripsi
Pada praktikum kali ini, Anda diminta untuk membuat model ANN untuk mengklasifikasi potensi seorang customer akan meninggalkan perusahaan Anda atau tidak. Istirlah populer dari fenomena ini disebut sebagai 'churn'. Tingkat churn yang tinggi (chrun rate) akan berdampak tidak baik bagi perusahaan.



## Pra Pengolahan Data


### Langkah 1 - Import Library


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Langkah 2 - Load Data


In [2]:
dataset = pd.read_csv('assets/docs/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [3]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Langkah 3 - Encoding Data Kategorikal


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [5]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### Langkah 4 - Encoding Kolom "Geography" dengan One Hot Encoder


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [7]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Langkah 5 - Split Data


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Langkah 6 - Scaling Fitur


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Membuat Model ANN


### Langkah 1 - Inisiasi Model ANN


In [10]:
ann = tf.keras.models.Sequential()

### Langkah 2 - Membuat Input Layer dan Hidden Layer Pertama


In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Langkah 3 - Membuat Hidden Layer Kedua


In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Langkah 4 - Membuat Output Layer


In [13]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training Model


### Langkah 1 - Compile Model (Menyatukan Arsitektur) ANN


In [14]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Langkah 2 - Fitting Model


In [15]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 3s 2ms/step - loss: 0.5664 - accuracy: 0.7699
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4947 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4612 - accuracy: 0.7993
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4431 - accuracy: 0.8060
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4344 - accuracy: 0.8098
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4285 - accuracy: 0.8105
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4248 - accuracy: 0.8121
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4215 - accuracy: 0.8161
Epoch 9/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4191 - accuracy: 0.8196
Epoch 10/100
250/250 [==============================] - 2s 7ms/step - loss: 0.4172 - accura

## Membuat Prediksi
Diberikan informasi sebagai berikut,

* Geography: France

* Credit Score: 600

* Gender: Male

* Age: 40 years old

* Tenure: 3 years

* Balance: $ 60000

* Number of Products: 2

* Does this customer have a credit card ? Yes

* Is this customer an Active Member: Yes

* Estimated Salary: $ 50000

Apakah customer tersebut perlu dipertahankan?

## Modelkan Data Baru dan Buat Prediksi


In [22]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 17ms/step
[[False]]


## Prediksi Dengan Data Testing


In [23]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Cek Akurasi dan Confusion Matrix


In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1521   74]
 [ 198  207]]


0.864